##### tiny,base,small,medium,large

In [ ]:
import speech_recognition as sr
import whisper
import numpy as np
from scipy import signal
import threading, time

model = whisper.load_model("tiny") #base, tiny, small, medium, large
r = sr.Recognizer()
mic = sr.Microphone()

transcribing = True
def transcribe_audio():
    global transcribing
    with mic as source:
        r.adjust_for_ambient_noise(source)
        print("Listening...")
        while transcribing:
            try:
                audio = r.listen(source, phrase_time_limit=12)
                audio_data = np.frombuffer(audio.get_raw_data(), dtype=np.int16)
                audio_data = audio_data.astype(np.float32) / 32767.0 
                resampled_audio = signal.resample(audio_data, int(len(audio_data) * (16000 / audio.sample_rate)))
                transcript = model.transcribe(resampled_audio)
                print("Transcription: ", transcript['text'])
            except sr.UnknownValueError:
                print("Unable to transcribe the audio.")
            except sr.RequestError as e:
                print(f"Error: {e}")
                
thread = threading.Thread(target=transcribe_audio)
thread.start()
# Wait for user input to stop transcription
input("Press Enter to stop transcription...")
transcribing = False
thread.join()

In [ ]:
#whisper openai
#https://analyzingalpha.com/openai-whisper-python-tutorial